# WELCOME TO THE NOTEBOOK
------------------

### Importing the Modules

In [ ]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import re
import string

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
nltk.download('stopwords')
nltk.download('vader_lexicon')


from collections import Counter

from matplotlib import pyplot as plt
from matplotlib import ticker
import seaborn as sns
import plotly.express as px

sns.set(style="darkgrid")

### Importing the Dataset

let's check the shape of the dataframe

let's select the needed columns for our project

change the type of some columns

### Picking out the tweet texts

### Removing URLs from tweets

### Converting all tweets to lowercase

### Removing punctuations

### Removing stopwords

In [ ]:
['covid','#coronavirus', '#coronavirusoutbreak', '#coronavirusPandemic', '#covid19', '#covid_19', '#epitwitter', '#ihavecorona', 'amp', 'coronavirus', 'covid19']


### let's create a big list of words out of all the tweets 

### put the Cleaned text in main dataframe

# Sentiment Analysis 

Getting the polarity scores for each tweet

### Labeling the scores based on the compound polarity value

### let's join two dataframes

### Plotting the sentiment score counts